**Astroinformatica I (Semester 1 2025)**
# Tutorial Session 8: Software Projects

*N. Hernitschek*


---
## Contents
* [Errors & Exceptions](#first-bullet)
* [Debugging](#second-bullet)
* [Testing](#third-bullet)
* [Exercise](#fourth-bullet)
* [Summary](#fifth-bullet)


## 1. Errors & Exceptions <a class="anchor" id="first-bullet"></a>

How exactly coding errors are handled depends on the programming language.
Python is an interpreted language, and syntax errors are found by the Python parser. Arrows mark the last parsed command which gave an error. See the following example:


In [4]:
#true = True
while True:   # True
    print('Hello world')

NameError: name 'true' is not defined

Exceptions, on the other hand, are not syntax errors. Exceptions are caught during runtime. See the following examples:



In [5]:
1/0


ZeroDivisionError: division by zero

In [6]:
'1' + 1

TypeError: can only concatenate str (not "int") to str

We can also add our own exception handling. See the fllowing example where we try to open a file which doesn't exist:




In [8]:
try:
   file = open('filenamethatdoesnotexist.txt')
except FileNotFoundError:
   print('No such file!')
   # create the file
   #...

No such file!


Also in the following code example we use exception handling:


In [5]:
def newfunction():
    raise NotImplementedError("Still need to write this code")
    print("Still need...")

newfunction()


NotImplementedError: Still need to write this code

One can also do various kinds of *type checking* like in the examples below:

In [10]:
s = input("Please enter an integer: ")  # s is a string
if not isinstance(s, int):
    try:
       print("Casting ", s, " to integer.")
       i = int(s)
    except ValueError:
        print("Input only integers!")
    #except FileNotFoundError:    




Please enter an integer: a
Casting  a  to integer.
Input only integers!


In [13]:
# Assert invariants

if i % 3 == 0:
    print(1)
elif i % 3 == 1:
    print(2)
else:
    assert i % 3 == 2
    print(3)

2


## 2. Debugging <a class="anchor" id="second-bullet"></a>


Testing and debugging are essential aspects of software development, 


After testing - discovering that a bug exists -, the next challenge is finding where the bug is in code. There are a few tools for this.

**Stack traces** are the messages the Python interpreter gives when there’s an uncaught exception (see examples above). At first, they may seem cryptic, but using them is pretty straight-forward. From bottom to top, the trace prints:

    the error type and message (e.g. SyntaxError, IndexError,...)
    the line in which the error happened
    the function in which the error happened, along with the filename and line number
    The filename, line number, and function that called the function with the error (and all the way up the stack)

However, this isn't always enough, e.g. when a function returns the wrong value, or you’re not sure if a loop is behaving properly. The quick fall-back for this is to insert **print statements** wherever you’re unsure of the value.

We try this in the following example with a broken average-taking function.



In [7]:

#folder = 'myfolder'
#file = 'myfile.lc'
#filename = folder + file
#file = open(filename)

#print(file) # myfoldermyfile.lc

#########

def foo(x):
    return 1/x

def bar(y):
    return foo(1-y)

bar(1)

ZeroDivisionError: division by zero

**Exercise:** try to debug and fix the above function.

There are a couple other problems with using `print()` for debugging:

* you might forget to remove a print statement
* it messes up unit tests
* it corrupts the useful output to the user with debugging output

For these reasons, it’s better to use the logging module to write debugging messages.

Using `logging.debug()` instead of `print()` lets you output the debugging statements to a file, instead of stdout. Here is an example from the python docs:

In [23]:
import logging
logging.basicConfig(filename='example_error.log',level=logging.DEBUG)
logging.debug('This message should go to the log file')
logging.info('So should this')
logging.warning('And this, too')
logging.error('really bad error')
print('done')
logging.getLoggerClass().root.handlers[0].baseFilename



done


'/home/nhernits/Documents/_uni/_Antofagasta/_teaching/2024/astroinformatica_I_2024_1/_tutorial_sessions/tutorial_8/example.log'

*Exercise:* Try it with the above example.


Creating logs this way will also automatically format the messages (which you can customize). This is a good way to provide debugging data to your users (including yourself), like warnings that their data is in the wrong format or that another function is better.



## 2.1 Using a Debugger


This is the more sophisticated way. You may have noticed that one annoyance with `print` or `log` is that you need to quit and restart Python whenever the function definition is changed.

Python provides several tools and techniques to help with debugging. One of the most common advanced debugging tool is `pdb`, Python's built-in debugger.
In the following, we will see how to use it.

To use `pdb`, you first need to import it, and add the line `pdb.set_trace()` where you'd like to start debugging. 



In the following example, we use `pdb.set_trace()` to set a breakpoint in our code. When executed, the program will pause at this line, allowing us to interactively inspect variables, execute code step by step, and diagnose the issue. When the execution is interrupted, we can use the following commands:

`p <variable>` for printing the value of a variable
    
`n` to execute the next line

`s` to step into a function call

`c` to continue execution

`!<code>` to run any line of Python (including changing values)
    

In [25]:
import pdb

def divide_numbers(a, b):
    pdb.set_trace()
    result = a / b
    return result

divide_numbers(10, 0)


def average(xs):
    pdb.set_trace()
    length = len(xs)
    acc = 0
    for i in range(1, length):
        acc = acc + xs[i]
    return acc/length


> /tmp/ipykernel_45652/2839550747.py(5)divide_numbers()
      3 def divide_numbers(a, b):
      4     pdb.set_trace()
----> 5     result = a / b
      6     return result
      7 

ipdb> !print(a)
10
ipdb> !a=20
ipdb> !print(a)
20
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


ZeroDivisionError: division by zero


Note: sometimes print debugging is still useful, like in a loop if you don't want to stop every iteration, but want to see the values as they come up.
    
More on the `pdb` debugger can be found in the documentation:

https://docs.python.org/2/library/pdb.html

## 3. Testing <a class="anchor" id="third-bullet"></a>


Testing plays a vital role in ensuring that our code behaves as expected and meets the specified requirements. By writing tests, we can catch bugs early, validate our code's functionality, and facilitate code maintenance and refactoring. Python provides several testing frameworks, including the built-in `unittest` module and the third-party `pytest` library, which offers various features to make testing efficient and comprehensive. 



## 3.1 The `unittest` module


The `unittest` module is part of Python's standard library and provides a framework for writing and running tests.  To get started with `unittest`, let's consider an example where we want to test a function that adds two numbers. We can create test cases by subclassing the `unittest.TestCase` class, which provides useful assertion methods for checking the expected behavior of our code:

In [27]:
import unittest

def add_numbers(a, b):
    return a + b#+0.1

class TestAddNumbers(unittest.TestCase):
    def test_add_numbers(self):
        result = add_numbers(2, 3)
        self.assertEqual(result, 5)

#if __name__ == "__main__": 
#    unittest.main() #you can execute it like that outside of Jupyter

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

F
FAIL: test_add_numbers (__main__.TestAddNumbers)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_45652/2618087838.py", line 9, in test_add_numbers
    self.assertEqual(result, 5)
AssertionError: 5.1 != 5

----------------------------------------------------------------------
Ran 1 test in 0.004s

FAILED (failures=1)



In the above example, we define a test case `TestAddNumbers` that inherits from `unittest.TestCase`. The `test_add_numbers` method tests that `add_numbers` function by asserting that the result of adding 2 and 3 should be equal to 5. We run the tests by executing `unittest.main()`.




## 3.2 The `pytest` framework



Pytest (http://pytest.org) is a popular testing framework in the Python community that makes it easy to write and run advanced tests. It provides several features such as test discovery, test parameterization, and plugins that make testing more efficient and productive. Pytest is easy to install, and it integrates well with other testing tools.

Let's extend our previous example using `pytest`.

First, you need to install pytest using pip:



In [21]:

! pip install pytest

Defaulting to user installation because normal site-packages is not writeable


Pytest discovers and runs tests by searching for files named `test_*.py` or `*_test.py? in your project directory and subdirectories. Let’s look at an example test using pytest:





In [20]:

# please copy the code below to a file called
# test_example.py

import pytest

def add_numbers(a, b):
    return a + b

def test_add_numbers():
    assert add_numbers(2, 3) == 5
    assert add_numbers(5, 7) == 12  

<module 'pytest' from '/home/nhernits/.local/lib/python3.10/site-packages/pytest/__init__.py'>

In the above example, we define a function `add_numbers`. We use the `assert` statement to check if the result of adding 2 and 3 is equal to 5.

Simply copy the above code into a file `test_example.py` your project directory, then go to the command line (terminal, console) to execute the following command in your project directory in order to run tests in `pytest`:


In [ ]:
pytest

`pytest` automatically discovers and runs all the test files, reporting the test results with detailed information. It also provides advanced features like fixtures, parameterization and test discovery, which enable you to write more complex and efficient tests.

## 4 Exercise <a class="anchor" id="fourth-bullet"></a>


If you have time left, create and try to debug and test more complex examples.


## Summary <a class="anchor" id="fifth-bullet"></a>

In this lession, we have learnt how to debug and test your code.

This knowledge will be very valuable once you start writing more complex code. In combination with comments and documentation, you can ensure code quality which saves you and other users from a lot of frustration.


